# COVID-19 Interactive Analysis

## Stay Home and Stay Safe!

### Do your part
1. __STAY__ at home
2. __KEEP__ a safe distance
3. __WASH__ hands often
4. __COVER__ your cough
5. __SICK?__ Call the helpline!


In [1]:
import pandas as pd
import ipywidgets as widgets
import plotly
import plotly.express as px
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import pandas as pd
import matplotlib.pyplot as plt
import folium
import plotly.graph_objects as go
import seaborn as sns
import numpy as np

In [2]:
# loading data right from John Hopkins GitHub Time Series Directory

death= pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed= pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered= pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')


In [3]:
# renaming the df column names to lowercase
country.columns = map(str.lower, country.columns)
confirmed.columns = map(str.lower, confirmed.columns)
death.columns = map(str.lower, death.columns)
recovered.columns = map(str.lower, recovered.columns)

# changing province/state to state and country/region to country
confirmed = confirmed.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered = recovered.rename(columns={'province/state': 'state', 'country/region': 'country'})
death = death.rename(columns={'province/state': 'state', 'country/region': 'country'})
country = country.rename(columns={'country_region': 'country'})

In [4]:
# total number of confirmed, death and recovered cases
confirmed_total = int(country['confirmed'].sum())
deaths_total = int(country['deaths'].sum())
recovered_total = int(country['recovered'].sum())
active_total = int(country['active'].sum())

## Current Statistics:

In [14]:
print("Confirmed cases: %d" %confirmed_total)
print("Deaths: %d" %deaths_total)
print("Recovered cases: %d" %recovered_total)
print("Active cases: %d" %active_total)


Confirmed cases: 1289380
Deaths: 70590
Recovered cases: 270372
Active cases: 625105


# Statistics of Top 'n' Worst Hit Countries in terms of Confirmed Cases

'n' represents the number of countries. You can enter a number of your choice in the blank and see the variations.

In [15]:
#Visualising Worst hit countries
sorted_country = country.sort_values('confirmed', ascending= False)

In [7]:
#Highlighting columns
def highlight_col(x):
    r = 'background-color: red'
    p = 'background-color: purple'
    g = 'background-color: grey'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = p
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    return df1

In [8]:
def show_latest_cases(n):
    n = int(n)
    return sorted_country.head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='10')

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

<function __main__.show_latest_cases(n)>

# Visualising the Top 'n' Worst Hit Countries in terms of Confirmed Cases

Use the slider to adjust the value of 'n'

In [17]:
def bubble_chart(n):
    fig = px.scatter(sorted_country.head(n), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst hit countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 700
    )
    fig.show()

interact(bubble_chart, n=10)

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

<function __main__.bubble_chart(n)>

# Visualising the Top 'n' Worst Hit Countries in terms of Deaths

Use the slider to define the value of 'n'

In [22]:
sorted_deaths = country.sort_values('deaths', ascending= False)
def deaths(n):
    fig= px.bar(
        sorted_deaths.head(n),
        x = "country",
        y = "deaths",
        title=str(n) +" Worst hit countries",
        color_discrete_sequence=["red"], 
        height=500,
        width=800
    )
    fig.show()

interact(deaths,n=10)
    

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

<function __main__.deaths(n)>

# Country Wise Statistics

Enter the name of the country to view number of confirmed cases and deaths reported.

In [11]:
def plot_cases_of_a_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]
    
    df_list = [confirmed, death]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
    
    fig.update_layout(
        title="COVID 19 cases of " + country,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();

In [12]:
interact(plot_cases_of_a_country, country='World')

interactive(children=(Text(value='World', description='country'), Output()), _dom_classes=('widget-interact',)…

<function __main__.plot_cases_of_a_country(country)>

# A World Map showing the progression of COVID-19

In [25]:
world_map = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start=2, max_zoom = 6, min_zoom = 2)


for i in range(0,len(confirmed)):
    folium.Circle(
        location=[confirmed.iloc[i]['lat'], confirmed.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(confirmed.iloc[i,-1]+1.00001)))+0.2)*50000,
        color='red',
        fill_color='indigo',
        tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death.iloc[i,-1]/(confirmed.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)

world_map

# National Helpline Number in India-
1. 1075 (toll free)
2. +91-11-23978046


In [ ]:
#voila Covid-19_Dashboard.ipynb --ExecutePreprocessor.timeout=180